In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit.opflow import PauliSumOp, AbelianGrouper
from qiskit.circuit.library import EfficientSU2

from qiskit.algorithms import VQE, NumPyMinimumEigensolver, NumPyEigensolver
from qiskit.algorithms.optimizers import SPSA

from qiskit.providers.aer import QasmSimulator
from qiskit.test.mock import FakeBurlington
from qiskit import transpile, QuantumCircuit

from qiskit.providers.ibmq import IBMQ

import mthree

In [ ]:
# load account
IBMQ.load_account()

In [ ]:
# provider = IBMQ.get_provider(hub='ibm-q-internal', group='near-time')
provider = IBMQ.get_provider(hub='ibm-q', group='open')

In [ ]:
provider.backends()

In [ ]:
# define backends
ibmq_qasm_backend = provider.get_backend('ibmq_qasm_simulator')
qasm_backend = QasmSimulator(shots=8192)
noisy_backend = FakeBurlington()
real_backend = provider.get_backend('ibmq_lima')

In [ ]:
# select backend
backend = qasm_backend
# backend = noisy_backend
# backend = real_backend

# set number of shots
backend.shots = 8192

# 0. Circuit Sampling

In [ ]:
def raw_sample(circuit, backend=backend, optimization_level=3):
    
    # transpile circuit
    qc = transpile(circuit, backend, optimization_level=optimization_level)
    
    # run circuit
    job = backend.run(qc)
    counts = job.result().get_counts()
    
    # evaluate probabilities
    shots = sum(counts.values())
    probabilities = {b: c/shots for b, c in counts.items()}
    return probabilities
    

# instantiate and calibrate mitigation scheme
mit = mthree.M3Mitigation(backend)
mit.cals_from_system()
# mit.cals_from_system(method='independent')  # use this to speed up noisy simulation
    
def mitigated_sample(circuit, backend=backend, mit=mit, optimization_level=3):
    
    # transpile circuit
    qc = transpile(circuit, backend, optimization_level=optimization_level)
    
    # determine final qubit mappings
    maps = mthree.utils.final_measurement_mapping(qc)
    
    # run circuit
    job = backend.run(qc)
    counts = job.result().get_counts()
        
    # mitigate shots
    quasi_probabilities = mit.apply_correction(counts, maps)    
    return quasi_probabilities

# choose sampling function
sample = raw_sample
# sample = mitigated_sample

In [ ]:
circuit = QuantumCircuit(3)
circuit.h(0)
circuit.cx(0,1)
circuit.cx(1, 2)
circuit.measure_all()
circuit.draw()

In [ ]:
# raw sampling without noise
raw_sample(circuit, qasm_backend)

In [ ]:
# raw sampling with noise
raw_sample(circuit, backend)

In [ ]:
# mitigating sampling with noise
mitigated_sample(circuit, backend)

# 1. Estimation of Expectation Values and VQE

### 1.1 Preliminaries
- `PauliSumOp`
- `AbelianGrouper`
- Deriving measurement bases

In [ ]:
# define Hamiltonian
H = PauliSumOp.from_list([('XXI', 1), ('XII', 2), ('YIY', 3), ('ZZI', 4), ('XIY', 5)])

# second test Hamiltonian
# H = PauliSumOp.from_list([('ZZI', 1), ('ZII', 2), ('ZIZ', 3), ('IZZ', 4)])

print(H)

In [ ]:
# group commuting Pauli terms 
grouper = AbelianGrouper()
groups = grouper.convert(H)
print('-----------')
for group in groups.oplist:
    print(group)
    print('-----------')

In [ ]:
# derives measurement basis from group of commuting Pauli terms
for group in groups:
    basis = ['I']*group.num_qubits
    for pauli_string in group.primitive.paulis:
        for i, pauli in enumerate(pauli_string):
            p = str(pauli)
            if p != 'I':
                if basis[i] == 'I':
                    basis[i] = p
                elif basis[i] != p:
                    raise ValueError('PauliSumOp contains non-commuting terms!')
    print(basis)

### 1.2 Estimator class

In [ ]:
class Estimator():
    
    def __init__(self, observable, circuit, callback=None):
        """ Instantiate estimator for given observable and circuit. """
        
        # store problem
        self._observable = observable
        self._circuit = circuit
        
        # group Pauli terms
        self._grouper = AbelianGrouper()
        self._groups = self._grouper.convert(self._observable).oplist
        
        # derive measurement bases
        self._bases = []
        for group in self._groups:
            self._bases += [self._get_measurement_basis(group)]
            
        # construct circuits with post-rotations
        self._circuits = []
        for basis in self._bases:
            self._circuits += [self._append_post_rotation(self._circuit, basis)]
            
        # store callback
        self._callback = callback
        
    def _get_measurement_basis(self, observable):
        """ Derive measurement basis from observable or raise exception in case of non-commuting terms. """
                
        basis = ['I']*observable.num_qubits
        for pauli_string in observable.primitive.paulis:
            for i, pauli in enumerate(pauli_string):
                p = str(pauli)
                if p != 'I':
                    if basis[i] == 'I':
                        basis[i] = p
                    elif basis[i] != p:
                        raise ValueError('PauliSumOp contains non-commuting terms!')
        return basis
    
    def _append_post_rotation(self, circuit, basis):
        """ Append post rotation to circuit to measure in given basis. """
        
        new_circuit = circuit.copy()
        for i, pauli in enumerate(basis):
            if pauli == 'X':  # H @ X @ H = Z
                new_circuit.h(i)
            if pauli == 'Y':  # S^dag @ H @ Y @ H @ S = Z
                new_circuit.s(i)
                new_circuit.h(i)
        new_circuit.measure_all()
        return new_circuit
    
    def estimate(self, param_values=None):
        """ Estimate expectation value of given observable in state corresponding to given parameter values. """

        if param_values is None and self._circuit.num_parameters > 0 or\
            len(param_values) != self._circuit.num_parameters:
            raise ValueError('Wrong number of parameters!')
        
        value = 0        
        for group, circuit in zip(self._groups, self._circuits):
            value += self._estimate_group(group, circuit, param_values)        
            
        if self._callback:
            self._callback(np.real(value))
            
        return np.real(value)
        
    def _estimate_group(self, group, circuit, param_values):
        """ Estimate expectation value for group of commuting terms that can be measured in the same basis. """
        
        probabilities = sample(circuit.bind_parameters(param_values), backend)
        
        value = 0
        for (pauli, coeff) in zip(group.primitive.paulis, group.primitive.coeffs):
            val = 0
            p = str(pauli)
            for b, prob in probabilities.items():
                val += prob * np.prod([(-1)**(b[i] == '1' and p[i] != 'I') for i in range(len(b))])

            value += coeff * val
    
        return value

In [ ]:
# define parametrized circuit
circuit = EfficientSU2(H.num_qubits, entanglement='linear', reps=1)
circuit.decompose().draw()

In [ ]:
# instantiate estimator
estimator = Estimator(H, circuit, lambda x: print(f'objective = {x}'))

In [ ]:
# test on random parameters
theta = np.random.rand(12)
estimator.estimate(theta)

### 1.3 Classical baseline

In [ ]:
exact_solver = NumPyMinimumEigensolver()
exact_result = exact_solver.compute_minimum_eigenvalue(H)
print(exact_result)

### 1.4 Build custom VQE

In [ ]:
# instantiate optimizer
optimizer = SPSA(maxiter=100)

In [ ]:
x0 = np.random.rand(circuit.num_parameters)
result = optimizer.minimize(estimator.estimate, x0=x0)
print(result.fun)
print(result.x)

# store ground state parameters for later
ground_state_params = result.x

### 1.5 Compare to Qiskit VQE

In [ ]:
vqe = VQE(circuit, optimizer, quantum_instance=backend)

In [ ]:
result = vqe.compute_minimum_eigenvalue(H)
print(result)

### 1.6 Integrate in Qiskit

Implement Qiskit's `MinimumEigensolver` interface to reuse custom algorithm in Qiskit Applications stack.

In [ ]:
from qiskit.algorithms import MinimumEigensolver, VQEResult

class CustomVQE(MinimumEigensolver):
    
    def __init__(self, circuit, optimizer):
        self._circuit = circuit
        self._optimizer = optimizer
        
    def compute_minimum_eigenvalue(self, operator, aux_operators=None):
                
        # run optimization
        estimator = Estimator(H, self._circuit)        
        x0 = np.random.rand(self._circuit.num_parameters)
        res = self._optimizer.minimize(estimator.estimate, x0=x0)

        # evaluate auxilliary operators if given
        aux_operator_eigenvalues = None
        if aux_operators is not None:
            aux_operator_eigenvalues = []
            for aux_estimator in [ Estimator(aux_op, self._circuit) 
                                      for aux_op in aux_operators ]:
                aux_operator_eigenvalues += [aux_estimator.estimate(res.x)]
        
        # populate results
        result = VQEResult()
        result.aux_operator_eigenvalues = aux_operator_eigenvalues
        result.cost_function_evals = res.nfev
        result.eigenstate = None
        result.eigenvalue = res.fun
        result.optimal_parameters = res.x
        result.optimal_point = res.x
        result.optimal_value = res.fun
        return result

In [ ]:
custom_vqe = CustomVQE(circuit, optimizer)
result = custom_vqe.compute_minimum_eigenvalue(H)
print(result)

# 2. State Fidelity

In [ ]:
class Fidelity():
    
    def __init__(self, circuit_1, circuit_2, callback=None):
        """ Instantiate fidelity estimator. """
        self._circuit_1 = circuit_1
        self._circuit_2 = circuit_2
        
        self._circuit_1.remove_final_measurements()
        self._circuit_2.remove_final_measurements()
        
        self._callback = callback
        
    def estimate(self, param_values_1, param_values_2):
        """ Estimate fidelity between the two states defined by 
        the given parameter values. """
        
        # bind parameter values and transpile circuit
        qc = self._circuit_1.bind_parameters(param_values_1)
        qc.append(self._circuit_2.bind_parameters(param_values_2).inverse(), 
                  range(self._circuit_2.num_qubits))
        qc.measure_all()
        
        # run circuit and get probabilities
        probabilities = sample(qc, backend)
        
        # estimate fidelity: |<0|U_1^dag U_2|0>|^2
        fidelity = np.minimum(1.0, np.maximum(0.0, probabilities.get('0'*qc.num_qubits, 0.0)))
        
        if self._callback:
            self._callback(fidelity)
        
        return fidelity

In [ ]:
# instantiate fidelity
fidelity = Fidelity(circuit, circuit, lambda x: print(f'fidelity = {x}'))

In [ ]:
# sample random parameters
x = np.random.rand(circuit.num_parameters)
y = np.random.rand(circuit.num_parameters)

In [ ]:
# interpolation between parameters
ts = np.linspace(0, 1)
fidelities = []
for t in ts:
    fidelities += [fidelity.estimate(x, x + t*(y-x))]

In [ ]:
plt.plot(ts, fidelities)
plt.xlabel('t')
plt.ylabel('fidelity')
plt.show()

# 3. Variational Quantum Deflation (VQD)
https://arxiv.org/abs/1805.08138

### 3.1 Classical Baseline

In [ ]:
exact_solver = NumPyEigensolver(k=2)
exact_result = exact_solver.compute_eigenvalues(H)
print(exact_result)

### 3.2 Define VQD Objective

In [ ]:
# set penalty weight for overlap term
penalty = 25

# define objective for VQD
def vqd_objective(param_values, 
                  energy=estimator.estimate, 
                  overlap=lambda x: fidelity.estimate(x, ground_state_params),
                  penalty=penalty):

    value = energy(param_values)
    value += penalty * overlap(param_values)
    return value

### 3.3 Run VQD

In [ ]:
# run optimization to get first excited state
result = optimizer.minimize(vqd_objective, np.random.rand(circuit.num_parameters))
print(result)

In [ ]:
# determine energy
estimator.estimate(result.x)

# determine overlap with ground state
fidelity.estimate(result.x, ground_state_params);

### Integrate into Qiskit

Implement Qiskit's `Eigensolver` interface to reuse custom algorithm in Qiskit Applications stack.

In [ ]:
from qiskit.algorithms import Eigensolver, EigensolverResult

class VQD(Eigensolver):
    
    def __init__(self, circuit, optimizer):
        self._circuit = circuit
        self._optimizer = optimizer
        
    def compute_eigenvalues(self, operator, aux_operators=None):
        
        # setup estimators
        estimator = Estimator(operator, self._circuit)
        fidelity = Fidelity(self._circuit, self._circuit)
        
        # compute groundstate
        vqe = CustomVQE(self._circuit, self._optimizer)
        vqe_result = vqe.compute_minimum_eigenvalue(operator)
        
        # compute first excited states
        objective = lambda x: vqd_objective(x, estimator.estimate, 
                                            lambda y: fidelity.estimate(y, vqe_result.optimal_parameters))
        
        x0 = np.random.rand(self._circuit.num_parameters)
        res = self._optimizer.minimize(objective, x0)
                
        # populate results
        result = EigensolverResult()
        result.eigenvalues = [vqe_result.eigenvalue, res.fun]        
        return result        

In [ ]:
vqd = VQD(circuit, optimizer)
result = vqd.compute_eigenvalues(H)
print(result)

# 4. Outlook: Quantum Computational Primitives

- We will move towards quantum computational primitives, powered by Qiskit Runtime, e.g., for sampling, estimation, etc.
- These will encapsulate error mitigation and other features to improve the ease of use.
- The complete Qiskit Algorithms & Applications stack will be extended accordingly over 2022.